<a href="https://colab.research.google.com/github/AtSourav/AE-w.-Bottleneck-Residual-Blocks/blob/main/AE_cifar10_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this notebook we shall use two custom python modules where we have defined some helper functions for plotting images, and more importantly some custom layers to implement residual blocks and in particular bottleneck residual blocks.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import metrics
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

# **Cloning the github repo with custom modules**

In [2]:
! git clone https://github.com/AtSourav/AE-w.-Bottleneck-Residual-Blocks

fatal: destination path 'AE-w.-Bottleneck-Residual-Blocks' already exists and is not an empty directory.


In [3]:
%cd "/content/AE-w.-Bottleneck-Residual-Blocks"

/content/AE-w.-Bottleneck-Residual-Blocks


In [4]:
import plotting_tools as plts
import Bottleneck_residual_blocks as resblock

# **Set-up, and dataset (cifar10) loading**

In [5]:
img_ht = 32
img_wd = 32

input_size = (img_ht,img_wd,3)
latent_dim = 256
batch_size = 128

initializer = initializers.HeNormal(seed=100)

In [6]:
ds = tfds.load('cifar10', split='train')
ds2= tfds.load('cifar10', split='test')

ds_to_np = list(ds.as_numpy_iterator())
ds2_to_np = list(ds2.as_numpy_iterator())

img_train = np.array([x['image'] for x in ds_to_np])
img_valid = np.array([x['image'] for x in ds2_to_np])

img_train = img_train/255
img_valid = img_valid/255

# **Building the model**

In [7]:
# we're using a combination of max and min pooling, min pooling in the skip connections in the residual blocks, max pooling otherwise

# intuitively, max pooling should be ideal for lighter images on a dark background, min pooling for the opposite scenario

encoder_input = keras.Input(shape=input_size)

x = resblock.conv2d_block(64, 3, initializer)(encoder_input)

x = resblock.conv2d_block(64, 3, initializer)(x)

# ------------------------------------------------------------------

x = resblock.conv2d_block(128, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(128, 2, 3, initializer, 'min', use_bn='False')(x)
        # the second argument is the compress-ratio for the bottleneck, the third argument is the kernel_size

x = resblock.bottleneck_residual_conv2D_block(128, 2, 3, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(128, 2, 3, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(128, 2, 3, initializer, 'min', use_bn='False')(x)

# ------------------------------------------------------------------

x = resblock.conv2d_block(256, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(256, 2, 3, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(256, 2, 3, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(256, 2, 3, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(256, 2, 2, initializer, 'min', use_bn='False')(x)

# -------------------------------------------------------------------

x = resblock.conv2d_block(512, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(512, 2, 3, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(512, 2, 3, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn='False')(x)

# -------------------------------------------------------------------

x = resblock.conv2d_block(1024, 1, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(1024, 2, 1, initializer, 'min', use_bn='False')(x)

# ...................................................................


x = layers.Flatten()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.ReLU()(x)

z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_block (conv2d_block  (None, 30, 30, 64)        2048      
 )                                                               
                                                                 
 conv2d_block_1 (conv2d_blo  (None, 28, 28, 64)        37184     
 ck)                                                             
                                                                 
 conv2d_block_2 (conv2d_blo  (None, 26, 26, 128)       74368     
 ck)                                                             
                                                                 
 bottleneck_residual_conv2d  (None, 24, 24, 128)       53504     
 _block (bottleneck_residua                                

In [8]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(2*latent_dim)(latent_input)
x = layers.ReLU()(x)

#x = layers.Dense(4*latent_dim)(x)
#x = layers.ReLU()(x)

x = layers.Dense(1*1*1024)(x)
x = layers.ReLU()(x)

x = layers.Reshape((1,1,1024))(x)

# ---------------------------------------------------

#x = resblock.conv2dtrans_block(1024, 2, initializer, strides=(2,2))(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(1024, 2, 2, initializer)(x)

# ---------------------------------------------------

x = resblock.conv2dtrans_block(512, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(512, 2, 3, initializer)(x)

# ---------------------------------------------------

x = resblock.conv2dtrans_block(512, 3, initializer, strides=(2,2))(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(512, 2, 3, initializer)(x)

# ---------------------------------------------------

x = resblock.conv2dtrans_block(256, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(256, 2, 3, initializer)(x)

x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)

# ---------------------------------------------------

x = resblock.bottleneck_residual_conv2D_block(256, 2, 2, initializer, 'min')(x)

# ---------------------------------------------------

x = resblock.conv2d_block(128, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(128, 2, 2, initializer, 'min')(x)

# ----------------------------------------------------

x = resblock.conv2d_block(64, 2, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(64, 2, 2, initializer, 'min')(x)

# -----------------------------------------------------

decoder_output = layers.Conv2D(3, 1, activation='sigmoid', padding='valid', kernel_initializer=initializer)(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 re_lu_46 (ReLU)             (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 re_lu_47 (ReLU)             (None, 1024)              0         
                                                                 
 reshape (Reshape)           (None, 1, 1, 1024)        0         
                                                                 
 bottleneck_residual_conv2_  (None, 2, 2, 1024)        2103

In [9]:
decoder_out = decoder(encoder(encoder_input))
AE = keras.Model(encoder_input, decoder_out, name='AE')

AE.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 encoder (Functional)        (None, 256)               7833024   
                                                                 
 decoder (Functional)        (None, 32, 32, 3)         13452291  
                                                                 
Total params: 21285315 (81.20 MB)
Trainable params: 21272771 (81.15 MB)
Non-trainable params: 12544 (49.00 KB)
_________________________________________________________________


# **Training**

In [12]:
K.clear_session()

In [ ]:
optim = keras.optimizers.Adam(learning_rate = 0.0005)
AE.compile(optimizer=optim,loss='mse')
history = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=10, steps_per_epoch=None)

Epoch 1/10
391/391 [==============================] - 112s 185ms/step - loss: 0.0468 - val_loss: 0.0746
Epoch 2/10
391/391 [==============================] - 63s 162ms/step - loss: 0.0379 - val_loss: 0.0588
Epoch 3/10
391/391 [==============================] - 63s 162ms/step - loss: 0.0358 - val_loss: 0.0398
Epoch 4/10
391/391 [==============================] - 63s 162ms/step - loss: 0.0337 - val_loss: 0.0369
Epoch 5/10
341/391 [=========================>....] - ETA: 7s - loss: 0.0335

In [11]:
optim = keras.optimizers.Adam(learning_rate = 0.0001)
AE.compile(optimizer=optim,loss='mse')
history2 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=10, steps_per_epoch=None)

Epoch 1/10
391/391 [==============================] - 77s 140ms/step - loss: 0.0204 - val_loss: 0.0202
Epoch 2/10
132/391 [=========>....................] - ETA: 33s - loss: 0.0202

KeyboardInterrupt: ignored